# Hybrid recommendation

In [40]:
#librairies
# classic Librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import shutil

#loading embeddings
import pickle

#sklearn utils
#from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neighbors import NearestNeighbors

#surprise
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split #really different from sklearn's ? 
from surprise import accuracy

In [41]:
# Options for cleaner display
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 1000)

In [42]:
# Load article metadata
data_articles = pd.read_csv("data/archive/articles_metadata.csv")
# Ensure ordering matches the embeddings file
data_articles = data_articles.sort_values("article_id").reset_index(drop=True)

# Load embeddings (shape ≈ [n_articles, 250])
with open("data/archive/articles_embeddings.pickle", "rb") as f:
    embeddings = pickle.load(f)

In [43]:
print("Shape de data_articles :", data_articles.shape)
print(data_articles.columns.tolist())
data_articles.head()

Shape de data_articles : (364047, 5)
['article_id', 'category_id', 'created_at_ts', 'publisher_id', 'words_count']


,article_id,category_id,created_at_ts,publisher_id,words_count
0,0,0,1513144419000,0,168
1,1,1,1405341936000,0,189
2,2,1,1408667706000,0,250
3,3,1,1408468313000,0,230
4,4,1,1407071171000,0,162


In [44]:
# optional renaming of columns
"""# 2.2. Renommage et transformation de colonnes
data_articles = data_articles.rename(columns={
    "category_id": "category",
    "publisher_id": "publisher",
    "created_at_ts": "created_at"
})

# Convertir created_at en datetime
data_articles["created_at"] = pd.to_datetime(data_articles["created_at"], unit="s", origin="unix")

# Garder uniquement les colonnes utiles
data_articles = data_articles[["article_id", "category", "publisher", "words_count", "created_at"]]

print("Après renommage :", data_articles.shape)
data_articles.head()
"""

'# 2.2. Renommage et transformation de colonnes\ndata_articles = data_articles.rename(columns={\n    "category_id": "category",\n    "publisher_id": "publisher",\n    "created_at_ts": "created_at"\n})\n\n# Convertir created_at en datetime\ndata_articles["created_at"] = pd.to_datetime(data_articles["created_at"], unit="s", origin="unix")\n\n# Garder uniquement les colonnes utiles\ndata_articles = data_articles[["article_id", "category", "publisher", "words_count", "created_at"]]\n\nprint("Après renommage :", data_articles.shape)\ndata_articles.head()\n'

In [45]:
# Embedding checker
# 3.1. Charger le pickle embeddings (numpy ndarray)
with open("data/archive/articles_embeddings.pickle", "rb") as f_in:
    embeddings = pickle.load(f_in)

# 3.2. Vérifier la forme
print("Type embeddings :", type(embeddings))
print("Shape embeddings :", embeddings.shape)
# embeddings doit être de shape (n_articles, 250)

# 3.3. Vérifier la correspondance entre embeddings et data_articles
#      On suppose que les embeddings sont **dans le même ordre** que les lignes de data_articles.
n_data_articles = data_articles.shape[0]
n_emb  = embeddings.shape[0]
print(f"Articles dans data_articles : {n_data_articles}, Lignes embeddings : {n_emb}")

if n_data_articles != n_emb:
    raise ValueError("Le nombre de lignes dans data_articles et dans embeddings ne correspond pas !"
                     " VÉRIFIE L’ORDRE DES ARTICLES.")


Type embeddings : <class 'numpy.ndarray'>
Shape embeddings : (364047, 250)
Articles dans data_articles : 364047, Lignes embeddings : 364047


In [46]:
# 4.1. Instancier NearestNeighbors
CF_RADIUS = n_data_articles  # nombre de voisins à rescanner pour le blending
nn_index = NearestNeighbors(n_neighbors=CF_RADIUS, metric="cosine", algorithm="auto")

# 4.2. Entraîner l’index sur l’ensemble des embeddings
nn_index.fit(embeddings)

print("Index CB prêt (NearestNeighbors).")


Index CB prêt (NearestNeighbors).


In [47]:
# 5.1. Charger l’instance Surprise picklée (algo_cf)
CF_MODEL_PATH = os.path.join("models_in_progress", "cf_model.pkl")

with open(CF_MODEL_PATH, "rb") as f_in:
    cf_algo = pickle.load(f_in)

# 5.2. Vérifier que cf_algo dispose bien de .predict, .train… etc.
print("Type cf_algo :", type(cf_algo))
# Exemple d’attribut attendu : cf_algo.__class__ doit être surprise.prediction_algorithms.matrix_factorization.SVD

Type cf_algo : <class 'surprise.prediction_algorithms.matrix_factorization.SVD'>


I don't need to reload dataset for CF since I have the model that's trained on it pickled

I can move on to reconstruct my functions

In [49]:
def build_user_profile(user_clicks):
    """
    user_clicks : liste d'article_id déjà vus (ex. [157541, 280367, 71301])
    Retourne un vecteur moyen des embeddings correspondants.
    Si aucun clic, renvoie un vecteur nul de même dimension.
    """
    if len(user_clicks) == 0:
        # Cold-start user : vecteur nul
        return np.zeros(embeddings.shape[1])

    # Trouver les indices dans data_articles pour chaque article_id
    idxs = data_articles.index[data_articles["article_id"].isin(user_clicks)].tolist()
    if len(idxs) == 0:
        # Aucun match (utilisateur a cliqué sur des articles hors data_articles)
        return np.zeros(embeddings.shape[1])

    user_embs = embeddings[idxs]
    return user_embs.mean(axis=0)


In [50]:
def score_cf_for_candidates(user_id, candidate_ids):
    """
    user_id : int
    candidate_ids : liste d'int (article_id)
    Retourne un numpy array de score CF brute : cf_algo.predict(user_id, iid).est
    """
    cf_scores = []
    for iid in candidate_ids:
        # on met r_ui=None car on ne connaît pas la vraie note
        pred = cf_algo.predict(uid=user_id, iid=iid, r_ui=None, verbose=False)
        cf_scores.append(pred.est)
    return np.array(cf_scores)


In [51]:
def normalize_minmax(array):
    """
    Ramène array dans [0,1] par un simple min-max scaling.
    Si array.min() == array.max(), on renvoie un vecteur constant à 0.5.
    """
    mn = array.min()
    mx = array.max()
    if mx > mn:
        return (array - mn) / (mx - mn)
    else:
        return np.full_like(array, 0.5, dtype=float)


In [52]:
def recommend_hybrid(user_id, user_clicks, k=10, alpha=0.5, total_candidates=CF_RADIUS):
    """
    Renvoie un DataFrame pandas des k articles recommandés pour user_id,
    en blendant le score CB (similarité cos) et le score CF (prediction SVD).
    
    user_id        : int
    user_clicks    : liste d'article_id déjà cliqués
    k              : nombre d’articles à retourner
    alpha          : poids du CF (0 <= alpha <= 1). ex. 0.5 pour 50% CF / 50% CB
    total_candidates : taille du pool initial de candidats CB
    
    Sortie : DataFrame contenant [
        article_id, category, publisher, words_count, created_at,
        score_cb, score_cf, score_hybrid
    ]
    """
    # ----- 1. Calculer le profil CB de l'utilisateur -----
    user_vec = build_user_profile(user_clicks)  # vecteur 250-d
    
    # ----- 2. Récupérer le pool initial via NN sur embeddings -----
    distances, indices = nn_index.kneighbors([user_vec], n_neighbors=total_candidates)
    cand_idxs = indices[0]                # indices dans data_articles/embeddings
    sims_cb = 1.0 - distances[0]          # cosinus similarity = 1 - distance
    
    # IDs des articles candidats
    candidate_ids = data_articles.iloc[cand_idxs]["article_id"].tolist()
    
    # ----- 3. Construire le DataFrame brut des candidats -----
    df_cand = pd.DataFrame({
        "article_id": candidate_ids,
        "score_cb": sims_cb
    })
    
    # ----- 4. Calculer le score CF brut (cf_algo.predict) -----
    raw_cf = score_cf_for_candidates(user_id, candidate_ids)
    df_cand["score_cf_raw"] = raw_cf
    
    # ----- 5. Normaliser le score CF en [0,1] -----
    df_cand["score_cf"] = normalize_minmax(df_cand["score_cf_raw"].values)
    
    # ----- 6. Calculer le score hybride -----
    df_cand["score_hybrid"] = alpha * df_cand["score_cf"] + (1 - alpha) * df_cand["score_cb"]
    
    # ----- 7. Trier par score_hybrid et prendre les top-k -----
    topk = (
        df_cand
        .sort_values("score_hybrid", ascending=False)
        .head(k)
        .merge(
            data_articles,
            on="article_id",
            how="left"
        )
    )
    
    # ----- 8. Sélection / ordre des colonnes à renvoyer -----
    return topk[[
        "article_id",
        "category_id",
        "publisher_id",
        "words_count",
        "created_at_ts",
        "score_cb",
        "score_cf",
        "score_hybrid"
    ]].reset_index(drop=True)


In [71]:
# Exemple : user_id=1234 a cliqué sur ces articles
test_user_id    = 10
test_user_clicks = [81937, 112847]  # historique des articles cliqués

# On appelle la fonction hybride
recs = recommend_hybrid(
    user_id=test_user_id,
    user_clicks=test_user_clicks,
    k=5,
    alpha=0.6,            # 60% CF / 40% CB
    total_candidates=n_data_articles
)

print("Recommandations hybrides pour user", test_user_id)
recs

Recommandations hybrides pour user 10


,article_id,category_id,publisher_id,words_count,created_at_ts,score_cb,score_cf,score_hybrid
0,83841,174,0,155,1507100284000,0.440159,0.912931,0.723822
1,157702,281,0,263,1507109598000,0.257656,1.000000,0.703062
2,156625,281,0,241,1507204625000,0.240647,1.000000,0.696259
3,96386,209,0,199,1507536073000,0.302086,0.944853,0.687746
4,119600,247,0,223,1507887563000,0.480693,0.804250,0.674827


nb de click par user sur un article / pondère par activité user lors de sa session (nb total de click par session)

par user's session  : \
nb total de click \
nb de click par article

temps passé sur chaque article \
nb de click sur un même lien

pour azure function deploy 

préparer une liste de user_id \
préparer des historiques différents

préparer sur papier les plans d'architecture imaginée/souhaitée/mise en place \
qu'est c eque je déploie en blob / qu'est ce que je déploie en aure Function